In [1]:
import pandas as pd

image_list = pd.read_csv("data/raw_image_list.csv")
image_list.head()

,User,Date,Isilon path,Single CZI,File Name,Experiment Type,QC?,Diff?,Cell Line,CL/CR#,...,Fluorescent Protein,Other Label,Drug/Treatment,Microscope,Objective,Dimension,MP?,Time-lapse,Exp. Details,Other
0,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001.czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,41.5hr long movie of lamin,cells not fed during imaging but look healthy
1,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(1).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 1",cells not fed during imaging but look healthy
2,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(2).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 2",cells not fed during imaging but look healthy
3,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(3).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 3",cells not fed during imaging but look healthy
4,CCH,2017.07.03,/aics/assay-dev/MicroscopyData/Caroline/2017/2...,Yes,20170703_C01_001(4).czi,AdditionalWorkup,NaN,iPSC,13.0,210,...,NaN,NaN,NaN,ZSD0,20x/0.80,3D,Yes,Yes,"41.5hr long movie of lamin, scene 4",cells not fed during imaging but look healthy


In [2]:
from pathlib import Path
import shutil

remote = (Path("/allen") / image_list.loc[1]["Isilon path"][1:] / image_list.loc[1]["File Name"]).resolve(strict=True)
print(f"Remote filepath: {remote}, Size in GB: {remote.stat().st_size / 1000 / 1000 / 1000}")

# local = Path(f"/tmp/{czi.name}")
# print(f"Beginning copy to: {local}")
# shutil.copy(str(czi), str(local))
# local.resolve(strict=True)
# print(f"Copy complete, located at: {local}")

Remote filepath: /System/Volumes/Data/allen/aics/assay-dev/MicroscopyData/Caroline/2017/20170703/20170703_C01_001(1).czi, Size in GB: 83.96012224


In [3]:
from timelapse_access.movies import generate_movie

generate_movie(remote, "/Users/jacksonb/Desktop/test.mp4", overwrite=True, series_range=slice(0, 100, 4), show_progress=True, C=0, fps=4)

/System/Volumes/Data/allen/aics/assay-dev/MicroscopyData/Caroline/2017/20170703/20170703_C01_001(1).czi


100%|██████████| 25/25 [16:17<00:00, 39.10s/it]


PosixPath('/Users/jacksonb/Desktop/test.mp4')